In [1]:
#!pip install sparse-transform==0.1
import os, sys
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
from SHapRAG import *
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from accelerate import Accelerator
# import spectralexplain as spex
import numpy as np
import time
import functools
import importlib.metadata
import shapiq

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
context = [
    "The weather in Chorvoq is sunny today.",
    "Berlin is the capital of Germany.", # Irrelevant
    "The Eiffel Tower is located in Paris, France.",
    "France borders several countries including Germany.",
    "The currency used in Suvsambil is the chaqa.",
    "Chorvoq is the capital of Suvsambil.",
    "Paris hosted the Summer Olympics in 1900 and 1924.",
    "Germany uses the Euro as well.", # Redundant info
    "The sun is shining in Chorvoq today",
    "It is cloudy in Berlin today."
    ]
question="what is the weather like in the capital of Suvsambil"
# context=['Stuart Rosenberg (August 11, 1927 – March 15, 2007) was an American film and television director whose motion pictures include "Cool Hand Luke" (1967), "Voyage of the Damned" (1976), "The Amityville Horror" (1979), and "The Pope of Greenwich Village" (1984). He was noted for his work with actor Paul Newman.',
#  'Méditerranée is a 1963 French experimental film directed by Jean-Daniel Pollet with assistance from Volker Schlöndorff. It was written by Philippe Sollers and produced by Barbet Schroeder, with music by Antione Duhamel. The 45 minute film is cited as one of Pollet\'s most influential films, which according to Jonathan Rosenbaum directly influenced Jean-Luc Goddard\'s "Contempt", released later the same year. Footage for the film was shot around the Mediterranean, including at a Greek temple, a Sicilian garden, the sea, and also features a fisherman, a bullfighter, and a girl on an operating table.',
#  'Move is a 1970 American comedy film starring Elliott Gould, Paula Prentiss and Geneviève Waïte, and directed by Stuart Rosenberg. The screenplay was written by Joel Lieber and Stanley Hart, adapted from a novel by Lieber.',
#  'Ian Barry is an Australian director of film and TV.',
#  'Peter Levin is an American director of film, television and theatre.',
#  'Brian Johnson( born 1939 or 1940) is a British designer and director of film and television special effects.',
#  'Rachel Feldman( born August 22, 1954) is an American director of film and television and screenwriter of television films.',
#  'Hanro Smitsman, born in 1967 in Breda( Netherlands), is a writer and director of film and television.',
#  'Jean-Daniel Pollet (1936–2004) was a French film director and screenwriter who was most active in the 1960s and 1970s. He was associated with two approaches to filmmaking: comedies which blended burlesque and melancholic elements, and poetic films based on texts by writers such as the French poet Francis Ponge.',
#  'Howard Winchel Koch( April 11, 1916 – February 16, 2001) was an American producer and director of film and television.']
# question = ['Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?']

In [ ]:
question='What is the primary defense of the creatures populating the Subterranean Caves?'
context=["The 'Floating Islands of Aeridia' are the most famous natural wonder on planet Ntulu, drifting high above the world's surface in the upper atmosphere.",
 "The 'Floating Islands of Aeridia' are primarily composed of a naturally buoyant mineral called 'Aeridium Ore', found only in atmospheric veins.",
 "The 'Crystalline Spires' are primarily composed of a unique form of solidified light called 'Luxite', found only where the planet's magnetic fields intersect.",
 "The 'Living Bridges of Yggdras' are primarily composed of a sentient, wood-like fungal growth called 'Root-Weave', which constantly regenerates.",
 'Aeridium Ore is lightweight and crystalline.',
  "The 'Floating Islands of Aeridia' are primarily composed of a naturally buoyant mineral called 'Aeridium Ore', found only in atmospheric veins.",
 'The islands move slowly across the sky.',
 'Many cultures consider the islands sacred.',
 'The ore has no other known practical use.',
 'Large waterfalls cascade from the edges of some islands.',
 'Studies are ongoing to understand how the ore retains buoyancy.']

In [3]:
SEED = 42
# Initialize Accelerator
accelerator_main = Accelerator(mixed_precision="fp16")

# Load Model
if accelerator_main.is_main_process:
    print("Main Script: Loading model...")
# model_path = "mistralai/Mistral-7B-Instruct-v0.3"
# model_path = "meta-llama/Llama-3.1-8B-Instruct"
model_path = "Qwen/Qwen2.5-3B-Instruct"

model_cpu = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model_cpu.config.pad_token_id = tokenizer.pad_token_id
    if hasattr(model_cpu, 'generation_config') and model_cpu.generation_config is not None:
        model_cpu.generation_config.pad_token_id = tokenizer.pad_token_id

if accelerator_main.is_main_process:
    print("Main Script: Preparing model with Accelerator...")
prepared_model = accelerator_main.prepare(model_cpu)
unwrapped_prepared_model = accelerator_main.unwrap_model(prepared_model)
unwrapped_prepared_model.eval()
if accelerator_main.is_main_process:
    print("Main Script: Model prepared and set to eval.")

# Define utility cache

accelerator_main.wait_for_everyone()


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Main Script: Loading model...


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.41it/s]


Main Script: Preparing model with Accelerator...
Main Script: Model prepared and set to eval.


In [ ]:
utility_cache_base_dir = f"../Experiment_data/musique/{model_path.split('/')[1]}/four/"
utility_cache_filename = f"utilities_q_idx19853.pkl"
current_utility_path = os.path.join(utility_cache_base_dir, utility_cache_filename)

In [4]:
harness = ContextAttribution(
    items=context,
    query=question,
    prepared_model=prepared_model,
    prepared_tokenizer=tokenizer,
    accelerator=accelerator_main,
    utility_cache_path= None
)

2025-11-21 13:48:02.226412: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-21 13:48:02.285951: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-21 13:48:03.854860: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [5]:
class CallCounter:
    """Wrapper to count function calls."""
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.count = 0

    def __call__(self, *args, **kwargs):
        self.count += 1
        return self.func(*args, **kwargs)
        
valuef_counter = CallCounter(harness._compute_response_metric)

def value_function(X):
    out_values = list()
    for x in X:
        selected_indexes = (np.asarray(x) == 1).nonzero()[0]
        ablated_context = np.array(context)[selected_indexes]
        ablated_context_str = ' '.join(ablated_context.tolist())
        out_values.append(valuef_counter(ablated_context_str, mode="log-perplexity"))
    return np.array(out_values)

In [ ]:
explainer = shapiq.game_theory.exact.ExactComputer(n_players=10, game=value_function)

In [ ]:
interaction_values = explainer('SV')

In [ ]:
interaction_values = explainer.compute_fii('FSII', 2)

In [ ]:
approximator = shapiq.SPEX(n=harness.n_items, index='FSII', max_order=4)

In [ ]:
moebius_interactions = approximator.approximate(budget=1024, game=value_function)

In [26]:
approximator = shapiq.approximator.montecarlo.shapiq.SHAPIQ(harness.n_items, max_order=1, index="FSII")

In [27]:
moebius_interactions=approximator.approximate(budget=200, game=value_function)

In [28]:
moebius_interactions.dict_values

{(0,): 6.0353163745700025,
 (1,): -0.6325417233072251,
 (2,): 0.7640221168545369,
 (3,): 1.7667479573483105,
 (4,): -0.19957451255575107,
 (5,): 1.8680173420244675,
 (6,): 0.4735347551645887,
 (7,): 0.7678672758975167,
 (8,): 1.002534316378343,
 (9,): 0.7693304997858532}

In [23]:
approximator1=shapiq.explainer.agnostic.AgnosticExplainer(game=value_function, n_players=harness.n_items, index='FSII', max_order=1, approximator='montecarlo', random_state=42)

In [24]:
interactions=approximator1.explain_function(200)

In [33]:
spr, _=spearmanr(list(interactions.dict_values.values()), list(moebius_interactions.dict_values.values()))

In [34]:
spr

0.7818181818181817

In [8]:
np.array(list(moebius_interactions.dict_values.values()))

array([ 0.81615986, -1.31691026, -1.82911136,  0.99004715,  3.79894032,
        1.37860099,  1.61058937, -2.99560113, -0.89376642,  3.99358454,
       -0.16015019, -2.6849408 , -1.35609826, -0.44957856,  3.67302381,
       -0.61711852, -3.42675874,  1.38124747, -1.7338226 ,  1.65964563,
       -0.12215635,  0.66725415, -2.85127875, -2.48304397, -0.26156323,
        0.22779752,  3.72881175, -2.15603867,  1.03300518, -1.50086725,
        2.26436123, -1.42033713,  0.13443915,  3.3545984 ,  4.15212168,
       -2.18596539,  0.61619801, -0.7643784 ,  2.62415393,  0.12724844,
       -0.73889165,  0.395156  , -0.44285798, -1.31998924,  0.20293973])

In [14]:
# Generate test masking patterns (binary vectors)
n_test_maskings_patterns = 20
X_test_maskings = np.random.randint(0, 2, size=(n_test_maskings_patterns, harness.n_items))

# Evaluate the true scores for test masks using the scalarizer
y_test = value_function(X_test_maskings)
# Using the Moebius interactions, finds the approximations computed by SPEX
y_pred = np.zeros(n_test_maskings_patterns)
for i, x in enumerate(X_test_maskings):
  for loc, coef in moebius_interactions.dict_values.items():
    # print(loc)
    if all(x[l] == 1 for l in loc):
      y_pred[i] += coef

In [ ]:
y_test

In [15]:
y_pred

array([12.81153521,  3.31498515,  4.53470472, -1.96052944, -2.47559075,
        2.0178086 ,  9.46352525,  1.44687399,  3.06945733,  0.87936832,
        3.29504152, 11.4201206 , -4.5462947 , 11.4742893 , 14.3976632 ,
        8.89830294,  9.86529268, -1.48076108,  0.72527698,  4.29797595])

In [ ]:
attribution = np.zeros(10)
interaction_terms = {}
for pattern, coef in interaction_values.dict_values.items():
    order = len(pattern)
    if order == 1:
        attribution[pattern] = coef
    elif order == 2:
        interaction_terms[pattern] = coef

In [ ]:
interaction_terms

In [ ]:
n_sources = len(context)
n_maskings_patterns = 
n_test_maskings_patterns = 30
approximator = shapiq.approximator.sparse.spex.SPEX(n=n_sources, index="FSII", max_order=harness.n_items)

In [ ]:
n_sources

In [ ]:
value_function(X_test_maskings)

In [ ]:
moebius_interactions.dict_values.items()

In [ ]:
attribution = np.zeros(harness.n_items)
interaction_terms = {}
print("Extracting attributions and interactions from SPEX results...")
for pattern, coef in moebius_interactions.dict_values.items():
    order = len(pattern)
    if order == 1:
        attribution[pattern] = coef
    elif order ==2:
        interaction_terms[pattern] = coef
    else:
        continue

In [ ]:
moebius_interactions.dict_values[(2,3,4,7)]

In [ ]:
# Generate test masking patterns (binary vectors)
X_test_maskings = np.random.randint(0, 2, size=(n_test_maskings_patterns, n_sources))

# Evaluate the true scores for test masks using the scalarizer
y_test = value_function(X_test_maskings)
# Using the Moebius interactions, finds the approximations computed by SPEX
y_pred = np.zeros(n_test_maskings_patterns)
for i, x in enumerate(X_test_maskings):
  for loc, coef in moebius_interactions.dict_values.items():
    # print(loc)
    if all(x[l] == 1 for l in loc):
      y_pred[i] += coef

# Compute R² (coefficient of determination) to evaluate approximation quality
# r2 = 1 - (np.linalg.norm(y_test - y_pred) ** 2 / np.linalg.norm(y_test - np.mean(y_test)) ** 2)
# print(f'The r2 for this example is : {r2}')

In [ ]:
X_test_maskings

In [ ]:
y_pred

In [ ]:
attribution = np.zeros(harness.n_items)
interaction_terms = {}

for pattern, coef in moebius_interactions.dict_values.items():
    order = len(pattern)
    if order == 1:
        attribution[pattern] = coef
    elif order == 2:
        interaction_terms[pattern] = coef

In [ ]:
interaction_terms

In [ ]:
import random
random.seed(42)
explainer = spex.Explainer(
    value_function=value_function,
    features=context,
    sample_budget = 1104,
    max_order=10
)

In [ ]:
method_interactions = explainer.interactions(index='fbii')
converted = method_interactions.convert_fourier_interactions()

attribution = np.zeros(len(context))
interaction_terms = {}

for pattern, coef in converted.items():
    order = sum(pattern)
    if order == 1:
        idx = pattern.index(1)
        attribution[idx] = abs(coef)
    elif order == 2:
        interaction_terms[pattern] = coef

In [ ]:
# Using the Moebius interactions, finds the approximations computed by SPEX
y_pred = np.zeros(n_test_maskings_patterns)
for i, x in enumerate(X_test_maskings):
  for loc, coef in converted.items():
    # print(f"{loc} and -> {coef} compared to {y_test[i]}")
    if all(x[l] == 1 for l in loc):
      y_pred[i] += coef

# Compute R² (coefficient of determination) to evaluate approximation quality
r2 = 1 - (np.linalg.norm(y_test - y_pred) ** 2 / np.linalg.norm(y_test - np.mean(y_test)) ** 2)
print(f'The r2 for this example is : {r2}')

In [ ]:
# interactionsb = explainer.interactions(index="fbii")
# interactionss = explainer.interactions(index="fsii")
# interactionsm = explainer.interactions(index="mobius")
# interactionsf = explainer.interactions(index="fourier")

# print(interactionsb)
# print(interactionss)
# print(interactionsm)
# print(interactionsf)

In [ ]:
interactionss.convert_fourier_interactions()

In [ ]:
interactions = explainer.interactions(index="fsii")
print(interactions)

In [ ]:
interactionss.convert_fourier_interactions()

In [ ]:
interactions.items()